# San Francisco Crime

Visualize crime spots and find a safe home.

---
### Table of Content

- [Icon Marker Map](#icon)
- [Circle Marker Map](#circle) 
- [Crime Spot Cluster Map](#cluster)
- [Crime Area Choropleth Map](#choropleth)

### Libraries

- `folium`, `geopandas`, `pandas`
---

#### Tip: Remember to zoom-in, zoom-out, hover, and click map points to see what will happen : ) - Yue

## Load Data

In [1]:
import pandas as pd
import folium 

In [2]:
path = 'https://cocl.us/sanfran_crime_dataset'
df = pd.read_csv(path, nrows=100)
df.head(3)

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010


In [3]:
loc=[37.77,-122.42]

## Icon Marker Map <a id="icon"></a>

In [4]:
sf_marker=folium.Map(location=loc,zoom_start=12)

for lag, lng, time in zip(df.Y, df.X, df.Time):
    folium.Marker(
        location=[lag, lng],
        radius=5,
        popup=time,
        icon=folium.Icon(color='red'),
        tooltip='click me for crime time'
    ).add_to(sf_marker)

sf_marker

## Circle Marker Map <a id="circle"></a>

In [5]:

sf_circle=folium.Map(location=loc,zoom_start=12)

for lag, lng, time in zip(df.Y, df.X, df.Time):
    folium.CircleMarker(
        location=[lag, lng],
        radius=6,
        tooltip='click me for time',
        popup=time,
        color='Yellow',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        marker_cluster=True
    ).add_to(sf_circle)


# sf_circle.save('circle-marker.html')
sf_circle 

## Crime Spot Cluster Map <a id="cluster"></a>

In [6]:
from folium import plugins
from folium.plugins import MarkerCluster

sf_cluster=folium.Map(loc,zoom_start=12)
cluster = plugins.MarkerCluster()

for lag,lng, time in zip(df.Y,df.X,df.Time):
    folium.Marker(
        location=[lag, lng],
        radius=5,
        popup=time,
        icon=folium.Icon(color='red'),
        tooltip='click me for crime time'
    ).add_to(cluster)

sf_cluster.add_child(cluster)

# sf_cluster.save('cluster-map.html')
sf_cluster

## Crime Area Choropleth Map <a id="choropleth"></a>

Get geo-info.

In [7]:
import geopandas as gpd

geo = gpd.read_file('./san-francisco.json') 
print(geo.head(2))  

   OBJECTID  DISTRICT COMPANY  \
0         1   CENTRAL       A   
1         2  SOUTHERN       B   

                                            geometry  
0  POLYGON ((-122.40532 37.80687, -122.40440 37.8...  
1  MULTIPOLYGON (((-122.38822 37.79000, -122.3880...  


In [8]:
incidents = df['PdDistrict'].value_counts().to_frame().reset_index() # add index
incidents.columns=['Neighborhood','Count']
incidents.head(5)

,Neighborhood,Count
0,SOUTHERN,20
1,MISSION,16
2,INGLESIDE,15
3,BAYVIEW,10
4,NORTHERN,9


In [9]:

crime_area = folium.Map(zoom_start=12, location =loc)

crime_area.choropleth(
    geo_data=geo,
    data=incidents,
    columns=['Neighborhood', 'Count'],
    key_on='feature.properties.DISTRICT',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='SanF'
)

folium.LayerControl().add_to(crime_area)

# crime_area.save('choropleth-map.html')
crime_area

/Applications/anaconda3/lib/python3.7/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


### Now you know where is safe : )
 The End.